# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
import os
import joblib

from azureml.train.automl import AutoMLConfig
from azureml.core import Model # Used to get model information
from azureml.data.dataset_factory import TabularDatasetFactory
import azureml

print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'udacity-project'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: quick-starts-ws-137945
Azure region: southcentralus
Subscription id: 610d6e37-4747-4a20-80eb-3aad70a55f43
Resource group: aml-quickstarts-137945


In [3]:
from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = "udacity-1"
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


I am using a Gaming dataset found on Kaggle. The dataset consists of 40 columns including "GameId" which is the unique identifier and "BlueWins" which indicates if Blue wins or Red.
I will be performing a binary classification as there are only two possible outcomes.

In [4]:
data_loc = "https://raw.githubusercontent.com/AishuDas/nd00333-capstone/master/starter_file/high_diamond_ranked_10min.csv"
ds = TabularDatasetFactory.from_delimited_files(data_loc)
data = ds.to_pandas_dataframe().dropna()


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

I refered to the following link for the input parameters. 
https://docs.microsoft.com/en-us/python/api/azureml-train-automl-client/azureml.train.automl.automlconfig.automlconfig?view=azure-ml-py
<br>
Upon reading the various parameters I chose the following <br>
1. Primary_metric as accuracy as Accuracy was the objective function <br>
2. I set Max concurrent iterations as 10 to parallize the automl model building process<br>
3. I set experiment timeout to 25 minutes so it doesn't go on building the model for a really long time.<br>
4. I set the task as classification<br>
5. I provided the label column name<br>
6. The training dataset<br>
7. The compute target to be used for computing<br>
8. Cross_validation as 5<br>
I left the rest of the parameters as their default values since they made sense for my run.<br>

In [5]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 25,
    "max_concurrent_iterations": 10,
    "primary_metric" : 'accuracy'}

computetarget = 'udacity'

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    compute_target=computetarget,
    task='classification',
    training_data=ds,
    label_column_name='blueWins',   
    n_cross_validations=5,
    **automl_settings
)

In [6]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.m

{'runId': 'AutoML_8f93ff38-cfe3-440f-a2cf-41a678f00ea8',
 'target': 'udacity',
 'status': 'Completed',
 'startTimeUtc': '2021-02-07T18:05:30.884672Z',
 'endTimeUtc': '2021-02-07T18:42:41.730715Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'udacity',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"610d6e37-4747-4a20-80eb-3aad70a55f43","resource_group":"aml-quickstarts-137945","workspace_name":"quick-starts-ws-137945","region":"southcentralus","compute_target":"udacity","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes":null,"featurizati

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [8]:
# Get best run and model
best_run, fitted_model = remote_run.get_output()
print(best_run)

# Get all metrics of the best run
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


Run(Experiment: udacity-project,
Id: AutoML_8f93ff38-cfe3-440f-a2cf-41a678f00ea8_156,
Type: azureml.scriptrun,
Status: Completed)
average_precision_score_weighted 0.8069067916154307
log_loss 0.5378598436902736
weighted_accuracy 0.7357280438205551
norm_macro_recall 0.4713519608205411
recall_score_micro 0.7357020960385384
AUC_micro 0.8091524094310486
AUC_macro 0.8092536849373655
matthews_correlation 0.47129191254689495
balanced_accuracy 0.7356759804102705
average_precision_score_micro 0.8064791989277549
precision_score_weighted 0.7358779323818071
f1_score_micro 0.7357020960385384
accuracy 0.7357020960385384
average_precision_score_macro 0.8066468531915187
AUC_weighted 0.8092536849373653
recall_score_macro 0.7356759804102705
f1_score_macro 0.7355771706999763
precision_score_micro 0.7357020960385384
precision_score_macro 0.7356159555340362
f1_score_weighted 0.7357212537738896
recall_score_weighted 0.7357020960385384
confusion_matrix aml://artifactId/ExperimentRun/dcid.AutoML_8f93ff38-cfe3-

In [16]:
#TODO: Save the best model
best_model = best_run.register_model(model_path='outputs/model.pkl', model_name='automl_best_model',tags={'Training context':'Auto ML'},properties={'Accuracy': best_run_metrics['accuracy']})

print(best_model)

joblib.dump(value=fitted_model, filename='model.pkl')

Model(workspace=Workspace.create(name='quick-starts-ws-137945', subscription_id='610d6e37-4747-4a20-80eb-3aad70a55f43', resource_group='aml-quickstarts-137945'), name=automl_best_model, id=automl_best_model:3, version=3, tags={'Training context': 'Auto ML'}, properties={'Accuracy': '0.7357020960385384'})


['model.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [12]:
# Download scoring file
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')

In [13]:
# Download environment file
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'conda_dependencies.yml')

In [18]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.environment import Environment

inference_config = InferenceConfig(entry_script='score.py', environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {'type': "automl_classification"}, 
                                               description = 'LOL Game Winner Prediction')

aci_service_name = 'lolpredict'
print(aci_service_name)

aci_service = Model.deploy(ws, aci_service_name, [best_model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

print(aci_service.scoring_uri)

print(aci_service.swagger_uri)

lolpredict
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://1cab3ccc-7644-4d42-be31-428f2dfd22a1.southcentralus.azurecontainer.io/score
http://1cab3ccc-7644-4d42-be31-428f2dfd22a1.southcentralus.azurecontainer.io/swagger.json


In [19]:
print(aci_service.get_logs())

2021-02-07T19:02:12,101611300+00:00 - gunicorn/run 
2021-02-07T19:02:12,102660000+00:00 - iot-server/run 
2021-02-07T19:02:12,137161000+00:00 - rsyslog/run 
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2021-02-07T19:02:12,158095400+00:00 - nginx/run 
rsyslogd

TODO: In the cell below, send a request to the web service you deployed to test it.

In [48]:
aci_service.update(enable_app_insights=True)

In [46]:
import json

testing_info = data.sample(3)
label = testing_info.pop('blueWins')
print(label)

request_json = json.dumps({'data': testing_info.to_dict(orient='records')})
print(request_json)


8393    1
5006    1
3176    1
Name: blueWins, dtype: int64
{"data": [{"gameId": 4519928092, "blueWardsPlaced": 24, "blueWardsDestroyed": 1, "blueFirstBlood": 1, "blueKills": 10, "blueDeaths": 7, "blueAssists": 9, "blueEliteMonsters": 2, "blueDragons": 1, "blueHeralds": 1, "blueTowersDestroyed": 0, "blueTotalGold": 18572, "blueAvgLevel": 7.2, "blueTotalExperience": 19279, "blueTotalMinionsKilled": 208, "blueTotalJungleMinionsKilled": 60, "blueGoldDiff": 1923, "blueExperienceDiff": 734, "blueCSPerMin": 20.8, "blueGoldPerMin": 1857.2, "redWardsPlaced": 14, "redWardsDestroyed": 2, "redFirstBlood": 0, "redKills": 7, "redDeaths": 10, "redAssists": 4, "redEliteMonsters": 0, "redDragons": 0, "redHeralds": 0, "redTowersDestroyed": 0, "redTotalGold": 16649, "redAvgLevel": 7.0, "redTotalExperience": 18545, "redTotalMinionsKilled": 205, "redTotalJungleMinionsKilled": 51, "redGoldDiff": -1923, "redExperienceDiff": -734, "redCSPerMin": 20.5, "redGoldPerMin": 1664.9}, {"gameId": 4463256999, "blueWard

In [47]:
import requests

headers = {'Content-type': 'application/json'}

#output = requests.post(aci_service.scoring_uri, request_json, headers=headers)
output = aci_service.run(request_json)

print(output)

{"result": [1, 1, 0]}


TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(aci_service.get_logs())
aci_service.delete()